In [12]:
import pandas as pd
import os
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
despesas = pd.read_csv('despesas.csv', sep=',', header=[0], encoding="utf-8")

In [17]:
resumo_despesa = despesas[['ano_exercicio', 'ds_municipio', 'ds_orgao', 'ds_despesa', 'tp_despesa', 'vl_despesa', 'ds_elemento', 'historico_despesa']]\
    .query('tp_despesa == "Empenhado" or tp_despesa == "Reforço" or tp_despesa == "Anulação"')

resumo_despesa.loc[resumo_despesa.tp_despesa == "Anulação", 'vl_despesa'] = resumo_despesa.vl_despesa * -1

resumo_despesa['ds_elemento'] = resumo_despesa['ds_elemento'].str[0:2]

In [18]:
lista_cc = [
    ('31', 'Pessoal_e_Encargos_Sociais'),
    ('32', 'Juros_e_Encargos_da_Divida'),
    ('33', 'Outras_Despesas_Correntes'),
    ('44', 'Investimentos'),
    ('45', 'Inversões_Financeiras'),
    ('46', 'Amortizacao_da_Divida')
]

lista_cc = pd.DataFrame(lista_cc)
lista_cc.columns = ['ds_elemento','desc']


In [19]:
resumo_despesa = pd.merge(resumo_despesa, lista_cc, how='inner')

In [21]:
credentials = service_account.Credentials.from_service_account_file(filename='GBQ.json',
                                                 scopes=['https://www.googleapis.com/auth/cloud-platform'])

In [23]:
resumo_despesa.to_gbq(credentials=credentials,
                      destination_table='despesas-378818.despesas.despesas_municipios2',
                      if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 2894.62it/s]


In [65]:
resumo_despesa.to_csv('loocker_despesa.csv', index=False)